## Integrantes

- **Estudiante 1:** Juan David Duarte Yara - 202215070
- **Estudiante 2:** Luisa Gabriela Hernández Pérez - 202114093
- **Estudiante 3:** Juan Sebastian Sanchez - 202121498
- Todos los estudiantes aportaron en las partes generales del notebook y en las diapositivas de coclusiones.



## 0. Carga de los datos. 

### 0.1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [1313]:
# Importaciones de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sys
import re
import string
import unicodedata

# Importaciones de joblib
from joblib import dump, load

# Importaciones de sklearn
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

# Clasificacion
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification

# Importaciones de statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, het_white

# Importaciones de nltk
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer, SnowballStemmer

# Instalación de paquetes adicionales
import contractions

# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Para la validación cruzada
from sklearn.model_selection import KFold

### 0.2. Lectura de archivos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [1314]:
# Se cargan los datos. 
datatrain=pd.read_csv('fake_news_spanish.csv', sep=';', encoding = 'utf-8', nrows=500)
datatest = pd.read_csv('fake_news_test.csv', sep=';', encoding = 'utf-8')

Podemos ver que hay 57,063 filas y 5 columnas de atributos en el conjunto de entrenamiento

In [1315]:
datatrain.shape

(500, 5)

Podemos ver que hay 1000 filas y 4 columnas de atributos en el conjunto de prueba

In [1316]:
datatest.shape

(1000, 4)

Veamos un pequeño sample de los datos

In [1317]:
datatrain.sample(10)

,ID,Label,Titulo,Descripcion,Fecha
64,ID,0,El exDAO dice que el exdirector de la Policía ...,El exdirector adjunto operativo Jenner López E...,14/12/2020
262,ID,1,Los sindicatos amenazan con movilizarse por el...,UGT y CCOO acusan a Pedro Sánchez y una parte ...,28/12/2020
410,ID,0,Arenas dice que la prisión permanente revisabl...,El vicesecretario de Política Autonómica y Loc...,14/01/2018
373,ID,0,Cristina Narbona hace un acto de fortaleza en ...,Pidió a los socialistas extremeños llenar la p...,19/05/2019
219,ID,0,Una senadora ultra colombiana afirma en un mit...,Fernanda Cabal acusa frente a Elías Bendodo al...,22/05/2023
361,ID,0,"La renuncia de Pablo Mónica García, entre bamb...",El líder de PP tomó la decisión de dar un paso...,20/07/2019
376,ID,1,Rajoy apela al Pacto de Toledo para bloquear l...,El Gobierno y el Partido Popular insisten en c...,13/03/2018
102,ID,1,Alba Rico: 'El discurso bélico al hablar del c...,El escritor y filósofo analiza en esta entrevi...,26/03/2020
333,ID,0,Negocios opacos y 'Operación Hidalgo': el tras...,"Marbella, la pieza 8 del caso Tándem, se abre ...",30/01/2020
233,ID,1,El anuncio de Sánchez para salud mental es un ...,En los PGE aparecen varias partidas destinadas...,21/05/2023


Revisemos que el tipo de dato sea el esperado

In [1318]:
datatrain.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

In [1319]:
datatrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           500 non-null    object
 1   Label        500 non-null    int64 
 2   Titulo       500 non-null    object
 3   Descripcion  500 non-null    object
 4   Fecha        500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


---


# 2. Entendimiento de los datos

## 2.1 Analisis descriptivo

Podemos ver que hay textos repetidos

In [1320]:
datatrain["Descripcion"].describe()


count                                                   500
unique                                                  500
top       Los colectivos auto-organizados y los especial...
freq                                                      1
Name: Descripcion, dtype: object

Es útil saber que tan largos son los textos

In [1321]:
datatrain["Descripcion"].apply(len).describe()

count     500.000000
mean      236.972000
std       182.653376
min        56.000000
25%       170.000000
50%       222.000000
75%       255.000000
max      3260.000000
Name: Descripcion, dtype: float64

Los textos tienen una longitud entre 33 y 14,153 caracteres, con una media de 243 caracteres. 

Queremos ver ahora la columna "label"

In [1322]:
datatrain["Label"].describe()

count    500.000000
mean       0.580000
std        0.494053
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Label, dtype: float64

Podemos ver que efectivamente la columna "Label" tiene valores entre 0 y 1.

Nos gustaría saber la proporción que hay entre noticias falsas y las que no

In [1323]:
datatrain["Label"].value_counts()

Label
1    290
0    210
Name: count, dtype: int64

Podemos ver que los datos están ligeramente desbalanceados

In [1324]:
datatrain.head(2)


,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023


In [1325]:
datatrain[["Titulo", "Descripcion"]].head(2)

,Titulo,Descripcion
0,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...
1,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...


Estadisticas de cada registro del tamaño del texto concatenado entre 
- *Conteo total:* titulo + descripcion
- *Conteo titulo:* titulo 
- *Conteo descripcion:*  descripcion
- *Min* : palabra mas corta de cada registro
- *Max* : palabra mas larga de cada registro


In [1326]:
textos = datatrain.copy()
textos["Message"] = (textos["Titulo"] + " " + textos["Descripcion"]).astype(str)

textos["Conteo total"] = textos["Message"].str.len()
textos["Conteo titulo"] = textos["Titulo"].str.len()
textos["Conteo descripcion"] = textos["Descripcion"].str.len()

textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Message']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Message']]

textos[["Conteo total", "Conteo titulo", "Conteo descripcion", "Max", "Min"]].describe()

,Conteo total,Conteo titulo,Conteo descripcion,Max,Min
count,500.000000,500.000000,500.000000,500.000000,500.00000
mean,331.248000,93.276000,236.972000,12.958000,1.04800
std,183.880119,21.631729,182.653376,1.860957,0.24044
min,127.000000,30.000000,56.000000,9.000000,0.00000
25%,264.750000,78.000000,170.000000,12.000000,1.00000
50%,313.000000,93.000000,222.000000,13.000000,1.00000
75%,358.000000,110.000000,255.000000,14.000000,1.00000
max,3365.000000,157.000000,3260.000000,20.000000,2.00000


## 2.2 Calidad de datos

In [1327]:
# Es recomendable que todos los pasos de limpieza y preparación se realicen sobre otro archivo.
data_t = datatrain.copy()

### Unicidad

Revisemos los duplicados por título 

In [1328]:
data_t.loc[datatrain.duplicated(subset= "Titulo", keep = False)].sort_values("Titulo").head(10)

,ID,Label,Titulo,Descripcion,Fecha


Revisemos ahora los duplicados por descripción 

In [1329]:
data_t.loc[datatrain.duplicated(subset= "Descripcion", keep = False)].sort_values("Descripcion").head(10)

,ID,Label,Titulo,Descripcion,Fecha


In [1330]:
data_t.duplicated(keep=False).sum()

np.int64(0)

Revisando el archivo pudimos notar que realmente los anteriores datos no son duplicados, sino que tienen ligeras diferencias entre su contenido, reafirmando que uno es una fake news y la otra no.

Ahora revisemos cuando la columna "Titulo", "Descripcción","Fecha" y "Label" son exactamente iguales.

In [1331]:
data_t.loc[datatrain.duplicated(subset= ["Descripcion","Titulo"], keep = False)].sort_values("Descripcion").head(10)

,ID,Label,Titulo,Descripcion,Fecha


Todas estas instancias son exactamente iguales, por lo cual, son duplicados. Revisemos cuantos duplicados realmente hay.

In [1332]:
data_t.duplicated(subset=['Titulo', 'Descripcion','Fecha','Label'], keep=False).sum()

np.int64(0)

Borremos los duplicados

In [1333]:
data_t = data_t.drop_duplicates(subset=['Titulo', 'Descripcion','Fecha','Label'], keep='first')

Revisemos si el tamaño de los datos cambió. El cambio debe ser consistente con la cantidad de duplicados eliminados

In [1334]:
datatrain.shape

(500, 5)

In [1335]:
data_t.shape

(500, 5)

Revisemos si hay duplicados

In [1336]:
data_t.duplicated(subset=['Titulo', 'Descripcion','Fecha','Label'], keep=False).sum()

np.int64(0)

### Completitud

In [1337]:
# Se observa que hay ausencias en la columna titulo
data_t.isnull().sum()

ID             0
Label          0
Titulo         0
Descripcion    0
Fecha          0
dtype: int64

In [1338]:
# Como solo son 16 instancias de más de 57,000 podemos borrar estos datos
data_t = data_t.dropna(subset=["Titulo"])

In [1339]:
# Se observa que ya no hay ausencias en la columna titulo
data_t.isnull().sum()

ID             0
Label          0
Titulo         0
Descripcion    0
Fecha          0
dtype: int64

### Validez

Podemos confirmar que solo existen los labels de fake news y no fake news.

In [1340]:
pd.DataFrame(data_t["Label"].unique())

,0
0,1
1,0


### Consistencia

Revisemos el formato de las fechas

In [1341]:
data_t["Fecha"].isna().sum()


np.int64(0)

In [1342]:
data_t['Nueva_Fecha'] = pd.to_datetime(data_t['Fecha'], format="%d/%m/%Y", errors ='coerce')
data_t['Nueva_Fecha'].sample(10)

72    2019-08-19
146   2018-06-24
132   2022-01-29
485   2021-01-27
88    2022-09-01
37    2018-10-23
366   2022-04-18
209   2021-10-04
305   2019-02-10
477   2022-04-14
Name: Nueva_Fecha, dtype: datetime64[ns]

In [1343]:
# Miramos los registros que no pudieron convertirse a DateTime. Vemos que todos se convirtieron satisfactoriamente a datetime y no hay ningun NaT
data_t.loc[data_t.Nueva_Fecha.isna(), ['Fecha','Nueva_Fecha']] 

,Fecha,Nueva_Fecha


In [1344]:
#Revisemos los tipos de datos
data_t.dtypes

ID                     object
Label                   int64
Titulo                 object
Descripcion            object
Fecha                  object
Nueva_Fecha    datetime64[ns]
dtype: object

# 3 Preparación de datos

## 3.0 Limpieza del texto

Se calcula una unica columna con texto que es la concatenacion de Titulo + " " + Descripcion en cada registro.

Tambien se eliminan las colunas que no se van a usar como la Fecha, Titulo, Descripcion y ID


In [1345]:
textos = datatrain.copy()
textos["message"] = (textos["Titulo"] + " " + textos["Descripcion"]).astype(str)
textos = textos.drop(columns=["ID", "Titulo", "Descripcion", "Fecha"])

textos.head(4)

,Label,message
0,1,'The Guardian' va con Sánchez: 'Europa necesit...
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild..."


## 3.1 Procesamiento de texto
Ahora tenemos que procesar el texto para que pueda utilizarlo el algoritmo de clasificación

In [1346]:
import nltk

# Descargar el conjunto de palabras de parada (stopwords) de nltk
nltk.download('stopwords')

# Descargar el tokenizer preentrenado 'punkt' para dividir texto en oraciones y palabras
nltk.download('punkt')

# Descargar el léxico WordNet para la lematización y otras tareas léxicas
nltk.download('wordnet')

# Descargar el Open Multilingual Wordnet (OMW) para soporte multilingüe en WordNet
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Las siguientes funciones nos servirán para hacer la limpieza, preprocesamiento y procesamiento de las palabras

In [1347]:
def aMinusculas(palabras):
    return [palabra.lower() for palabra in palabras]

def eliminarNumeros(palabras):
    return [re.sub(r'\d+', '', palabra) for palabra in palabras]

def eliminarPuntuacion(palabras):
    return [palabra for palabra in palabras if palabra not in string.punctuation]

def removerNoAscii(palabras):
    return [palabra for palabra in palabras if palabra.isascii()]

def eliminarStopwords(palabras):
    sw = set(stopwords.words('spanish'))
    return [palabra for palabra in palabras if palabra not in sw]

def aplicarStemmingYLematizacion(palabras):
    # Evidenciamos que brinda mejores resultados que PorterStemmer para español
    stemmer = SnowballStemmer('spanish')
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(stemmer.stem(palabra)) for palabra in palabras]

def preprocesar(palabras):
    palabras = aMinusculas(palabras)
    palabras = eliminarNumeros(palabras)
    palabras = eliminarPuntuacion(palabras)
    palabras = removerNoAscii(palabras)
    palabras = eliminarStopwords(palabras)
    return palabras

def procesar(texto):
    texto = contractions.fix(texto)
    palabras = word_tokenize(texto)
    palabras = preprocesar(palabras)
    palabras = aplicarStemmingYLematizacion(palabras)
    return " ".join(palabras)

A continuación podemos ver el resultado del procesamiento de los textos en la "message"

In [1348]:
textos["words"] = textos["message"].apply(procesar)


In [1349]:
textos.head(4)

,Label,message,words
0,1,'The Guardian' va con Sánchez: 'Europa necesit...,'the guardi va 'europ necesit apuest frut diar...
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,revel gobiern negoci mirel cambi otorg javi du...
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,'ahor nunc joan fust estatut valencian cumpl v...
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...",iglesi alient yoland erc eh bildu negoci 'rear...


In [1350]:
textos['words'].dropna()

0      'the guardi va 'europ necesit apuest frut diar...
1      revel gobiern negoci mirel cambi otorg javi du...
2      'ahor nunc joan fust estatut valencian cumpl v...
3      iglesi alient yoland erc eh bildu negoci 'rear...
4      puigdemont 'no ningun tragedi eleccion entrev ...
                             ...                        
495    alcanz absolut iglesi river ci macroencuest ci...
496    eurodiput pid europe 'inst alme revert madr ce...
497    alfons guerr trat reduc fug vot atac inici ver...
498    noven discurs lesm abrir judicial reproch nuev...
499    crisis hac aflor form sindical colect auto-org...
Name: words, Length: 500, dtype: object

In [1351]:
textos.shape

(500, 3)

In [1352]:
textos['words'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 500 entries, 0 to 499
Series name: words
Non-Null Count  Dtype 
--------------  ----- 
500 non-null    object
dtypes: object(1)
memory usage: 4.0+ KB


In [1353]:
X_data, y_data = textos['words'],textos['Label']
y_data = (y_data == '0').astype(int)
y_data

0      0
1      0
2      0
3      0
4      0
      ..
495    0
496    0
497    0
498    0
499    0
Name: Label, Length: 500, dtype: int64

## 3.2 NM-gramas
Implementamos NM-gramas con un rango de 1 a 3. 

In [1354]:
# Inicializar el vectorizador con reducción de características
bv = CountVectorizer(ngram_range=(1, 3), min_df=2, max_df=0.9)#, max_features=5000)

# Procesar "message" (sin convertir a denso)
bv_matrix_message = bv.fit_transform(textos["words"])
bv_lista_message = bv_matrix_message.toarray()

vocab_messages = bv.get_feature_names_out()
vocab_messages


array(['abandon', 'abascal', 'abertzal', ..., 'zapater', 'zaragoz', 'zon'],
      dtype=object)

In [1355]:
textos

,Label,message,words
0,1,'The Guardian' va con Sánchez: 'Europa necesit...,'the guardi va 'europ necesit apuest frut diar...
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,revel gobiern negoci mirel cambi otorg javi du...
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,'ahor nunc joan fust estatut valencian cumpl v...
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...",iglesi alient yoland erc eh bildu negoci 'rear...
4,0,Puigdemont: 'No sería ninguna tragedia una rep...,puigdemont 'no ningun tragedi eleccion entrev ...
...,...,...,...
495,1,Sánchez podría alcanzar la mayoría absoluta co...,alcanz absolut iglesi river ci macroencuest ci...
496,1,Eurodiputados piden a la Comisión Europea que ...,eurodiput pid europe 'inst alme revert madr ce...
497,0,Alfonso Guerra trata de reducir la fuga de vot...,alfons guerr trat reduc fug vot atac inici ver...
498,0,El noveno discurso de Lesmes para abrir el año...,noven discurs lesm abrir judicial reproch nuev...


In [1356]:
bv_df = pd.DataFrame(bv_lista_message, columns=bv.get_feature_names_out())
bv_df

,abandon,abascal,abertzal,abiert,abog,abord,abort,abre,abre puert,abril,...,vulner,web,xim,xim puig,xunt,xunt gobiern,yoland,zapater,zaragoz,zon
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1357]:
bv_df = pd.concat([textos.drop(columns=["message", "words"]), bv_df], axis=1)
bv_df

C:\Users\surface\AppData\Roaming\Python\Python311\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,Label,abandon,abascal,abertzal,abiert,abog,abord,abort,abre,abre puert,...,vulner,web,xim,xim puig,xunt,xunt gobiern,yoland,zapater,zaragoz,zon
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.4. Creación y mejora de los conjuntos de entrenamiento y validación

In [1358]:
# Se selecciona la variable objetivo
Y = bv_df['Label']
X = bv_df.drop(['Label'], axis=1)

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.3, random_state=0)
X_train

,abandon,abascal,abertzal,abiert,abog,abord,abort,abre,abre puert,abril,...,vulner,web,xim,xim puig,xunt,xunt gobiern,yoland,zapater,zaragoz,zon
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1359]:
print("NaN en X_train:", np.isnan(X_train).sum().sum())
print("Inf en X_train:", np.isinf(X_train).sum().sum())


NaN en X_train: 0
Inf en X_train: 0


In [1360]:
X_train = X_train.dropna()
X_train


,abandon,abascal,abertzal,abiert,abog,abord,abort,abre,abre puert,abril,...,vulner,web,xim,xim puig,xunt,xunt gobiern,yoland,zapater,zaragoz,zon
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1361]:
print("NaN en X_train:", np.isnan(X_train).sum().sum())
print("Inf en X_train:", np.isinf(X_train).sum().sum())

NaN en X_train: 0
Inf en X_train: 0


In [1362]:
print("NaN en Y_train:", Y_train.isna().sum())

Y_train = Y_train.dropna()

NaN en Y_train: 0


In [1363]:
X_train = X_train.loc[Y_train.index]
Y_train = Y_train.loc[X_train.index]


Balanceo de las clases

In [1364]:
smt = SMOTE()
X_smote, Y_smote = smt.fit_resample(X_train, Y_train)
X_smote

,abandon,abascal,abertzal,abiert,abog,abord,abort,abre,abre puert,abril,...,vulner,web,xim,xim puig,xunt,xunt gobiern,yoland,zapater,zaragoz,zon
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Se realiza estandarizacion de los datos

In [1365]:
scaler = StandardScaler()
X_smote = pd.DataFrame(scaler.fit_transform(X_smote), columns=vocab_messages)
X_validation = pd.DataFrame(scaler.fit_transform(X_validation), columns=vocab_messages)
X_smote

,abandon,abascal,abertzal,abiert,abog,abord,abort,abre,abre puert,abril,...,vulner,web,xim,xim puig,xunt,xunt gobiern,yoland,zapater,zaragoz,zon
0,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
1,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
2,-0.100251,-0.083414,-0.086711,-0.086711,8.124038,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
3,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
4,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
398,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
399,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938
400,-0.100251,-0.083414,-0.086711,-0.086711,-0.123091,-0.086711,-0.070711,-0.159719,-0.100251,-0.086711,...,-0.086711,-0.049938,-0.070711,-0.070711,-0.049938,-0.049938,-0.18135,-0.086711,-0.049938,-0.049938


---
---


# 4. Modelos

## 4.1 KNN - K vecinos mas cercanos
Juan David Duarte

In [1366]:
# Se selecciona la variable objetivo
Y = bv_df['Label']
X = bv_df.drop(['Label'], axis=1)

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y, test_size=0.3, random_state=0)

X_train1 = X_train1.loc[Y_train.index]
Y_train1 = Y_train1.loc[X_train.index]

smt = SMOTE()
X_smote1, Y_smote1 = smt.fit_resample(X_train1, Y_train1)

scaler = StandardScaler()
X_smote1 = pd.DataFrame(scaler.fit_transform(X_smote1), columns=vocab_messages)
#test1 = pd.DataFrame(scaler.fit_transform(X_test1validation), columns=vocab_messages)


In [1367]:
neigh = KNeighborsClassifier(n_neighbors=3) # arbol = DecisionTreeClassifier(criterion='entropy', random_state=0)
neigh = neigh.fit(X_train1, Y_train1) # arbol = arbol.fit(X_train,Y_train)
y_pred1 = neigh.predict(X_test1) # y_pred = arbol.predict(X_test)
print('Exactitud: %.2f' % accuracy_score(Y_test1, y_pred1))
print("Recall: {}".format(recall_score(Y_test1,y_pred1)))
print("Precisión: {}".format(precision_score(Y_test1,y_pred1)))
print("Puntuación F1: {}".format(f1_score(Y_test1,y_pred1)))

Exactitud: 0.60
Recall: 0.5280898876404494
Precisión: 0.7230769230769231
Puntuación F1: 0.6103896103896104


## 4.2 Arboles de decision

In [1368]:
## 